In [158]:
import pandas as pd

names=['Datum(UTC)','Nicht Erneuerbar (MW)', 'Erneuerbar (MW)', 'DE-LU Day Ahead EUR/MWh']
DayAheadPrices=pd.read_csv("Daten entsoe/energy-charts_Stromproduktion_und_Börsenstrompreise_in_Deutschland_2024.csv",skiprows=2,names=names)

In [159]:
#preis_menge=DayAheadPrices
DayAheadPrices['datum']=pd.to_datetime(DayAheadPrices['Datum(UTC)'])
DayAheadPrices=DayAheadPrices.set_index('datum')
preis_menge=DayAheadPrices

In [160]:
IKStrom=pd.read_csv("Daten IK8/2024 Hist Strom.csv",date_format='ISO8601',parse_dates=True,decimal='.')
IKStrom['last_changed dt']=pd.to_datetime(IKStrom['last_changed']).dt.floor('h')
IKStrom=IKStrom.set_index('last_changed dt')
ent_filter=['sensor.haushalt_positive_active_energy_total','sensor.haushalt_negative_active_energy_total','sensor.waermepumpe_positive_active_energy_total','sensor.waermepumpe_negative_active_energy_total']
IKStrom=IKStrom[IKStrom['entity_id'].isin(ent_filter)]
IKStrom['state']=pd.to_numeric(IKStrom['state'])
IKStrom=IKStrom.groupby('entity_id').resample('h')['state'].agg(['min']).ffill()
IKStrom=IKStrom.reset_index().pivot(index=['last_changed dt'],columns='entity_id',values='min')
IKStrom=IKStrom.ffill()

In [164]:
preis_menge[['Haus H','Haus S','WP H','WP S']]=IKStrom.diff().shift(-1).fillna(0)
preis_menge['WP PV']=preis_menge['Haus H']-preis_menge['WP H']

In [165]:
preis_menge

,Datum(UTC),Nicht Erneuerbar (MW),Erneuerbar (MW),DE-LU Day Ahead EUR/MWh,Haus H,Haus S,WP H,WP S,WP PV
datum,,,,,,,,,
2023-12-31 23:00:00+00:00,2023-12-31T23:00+00:00,8378.54,41780.27,0.10,0.0,216.6,0.0,351.7,0.0
2024-01-01 00:00:00+00:00,2024-01-01T00:00+00:00,8284.44,41524.81,0.01,0.0,329.0,0.0,1831.0,0.0
2024-01-01 01:00:00+00:00,2024-01-01T01:00+00:00,8282.03,41348.67,0.00,0.0,260.0,0.0,395.2,0.0
2024-01-01 02:00:00+00:00,2024-01-01T02:00+00:00,8318.30,40365.41,-0.01,0.0,241.7,0.0,1730.2,0.0
2024-01-01 03:00:00+00:00,2024-01-01T03:00+00:00,8309.99,40235.53,-0.03,0.0,304.4,0.0,1339.2,0.0
...,...,...,...,...,...,...,...,...,...
2024-12-31 18:00:00+00:00,2024-12-31T18:00+00:00,21383.15,38916.54,67.77,0.0,1139.5,0.0,3385.2,0.0
2024-12-31 19:00:00+00:00,2024-12-31T19:00+00:00,18557.03,40027.98,35.56,0.0,350.7,0.0,1189.2,0.0
2024-12-31 20:00:00+00:00,2024-12-31T20:00+00:00,15779.81,40653.81,15.70,0.0,368.1,0.0,2621.2,0.0


In [120]:

#IKStrom.resample('last_changed dt')['state'].agg(['min'])
IKStrom.to_csv('test.csv')

In [150]:
IKStrom

entity_id,sensor.haushalt_negative_active_energy_total,sensor.haushalt_positive_active_energy_total,sensor.waermepumpe_negative_active_energy_total,sensor.waermepumpe_positive_active_energy_total
last_changed dt,,,,
2023-12-31 23:00:00+00:00,31494937.7,3474184.3,29938260.3,13184191.9
2024-01-01 00:00:00+00:00,31494937.7,3474400.9,29938260.3,13184543.6
2024-01-01 01:00:00+00:00,31494937.7,3474729.9,29938260.3,13186374.6
2024-01-01 02:00:00+00:00,31494937.7,3474989.9,29938260.3,13186769.8
2024-01-01 03:00:00+00:00,31494937.7,3475231.6,29938260.3,13188500.0
...,...,...,...,...
2024-12-31 18:00:00+00:00,44812648.8,5325151.4,42654022.2,19421109.0
2024-12-31 19:00:00+00:00,44812648.8,5326290.9,42654022.2,19424494.2
2024-12-31 20:00:00+00:00,44812648.8,5326641.6,42654022.2,19425683.4


In [53]:
pd.to_datetime(IKStrom['last_changed'])

0               2023-12-31 23:00:00+00:00
1               2024-01-01 00:00:00+00:00
2               2024-01-01 01:00:00+00:00
3               2024-01-01 02:00:00+00:00
4               2024-01-01 03:00:00+00:00
                       ...               
131873   2024-12-31 22:57:25.281000+00:00
131874   2024-12-31 22:58:27.306000+00:00
131875   2024-12-31 22:59:27.516000+00:00
131876   2024-12-31 09:16:21.357000+00:00
131877   2024-12-31 09:16:21.646000+00:00
Name: last_changed, Length: 131878, dtype: datetime64[ns, UTC]

In [51]:
IKStrom['last_changed'].apply(pd.to_datetime())

TypeError: to_datetime() missing 1 required positional argument: 'arg'